

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [66]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

def str_to_idx(s): 
  n = []
  for x in s:
    n.append(ord(x) - 96)
  return torch.tensor(n)

def out_to_str(o):
  idx = torch.argmax(o, dim=-1).view(o.shape[1])
  s = ''
  for i in idx:
    if i!=0:
      s+=(chr(i+96))
    else:
      s+=(' ')
  return s

class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)


In [67]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [71]:
import torch.nn.functional as F

class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(27, self.hidden_size, num_layers=2, batch_first=True)
    self.fc1 = torch.nn.Linear(in_features=self.hidden_size, out_features=27)
  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    hidden_state = torch.zeros(2, x.size(0), self.hidden_size).to(device)
    output, hidden_state = self.gru(x, hidden_state)
    output = self.fc1(output)
    return output,hidden_state

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    idx = str_to_idx(s)
    input = idx_to_onehot(idx).view(-1,len(s),27).to(device)
    output,_ = self.forward(input)
    #print(output,output.shape,output.sum(-1),output.sum(-1).shape)
    output_str = out_to_str(output)
    return output_str

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [69]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(100):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [73]:
import time
from torch.utils.data import DataLoader
start_time = time.time()

# TODO
model = GRUMemory(18).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
batch_size = 128
Epochs = 3
train_dataloader = DataLoader(ds, batch_size)
for i in range(Epochs):
  print(i)
  for x,y in train_dataloader:
    x = idx_to_onehot(x)
    y = idx_to_onehot(y)
    # x 64*15*27 y 64*15
    x,y = x.to(device),y.to(device)
    logits, _ = model(x)
    loss = criterion(logits[:,4:15,:], y[:,4:15,:])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(test_model(model),loss,time.time()-start_time) 
    

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

0
0.941983780411728 tensor(0.1667, device='cuda:0', grad_fn=<DivBackward1>) 13.169416427612305
1
0.9861198738170347 tensor(0.1185, device='cuda:0', grad_fn=<DivBackward1>) 26.200608253479004
2
0.9993714644877436 tensor(0.1013, device='cuda:0', grad_fn=<DivBackward1>) 39.10888671875
tests passed


In [156]:
test_model(model)

1.0

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [5]:

import torch
import random
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=2000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

vds = VariableDelayEchoDataset(max_delay=8, seq_length=20)

### Model

And the model.

In [60]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    self.gru = torch.nn.GRU(27, self.hidden_size, num_layers=8, batch_first=True)
    self.fc1 = torch.nn.Linear(in_features=self.hidden_size, out_features=27)

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    hidden_state = delays.unsqueeze(-1).repeat(8,1,hidden_size).float().to(device)
    output, hidden_state = self.gru(x, hidden_state)
    output = self.fc1(output)
    return output,hidden_state

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    idx = str_to_idx(s)
    input = idx_to_onehot(idx).view(-1,len(s),27).to(device)
    output,_ = self.forward(input,torch.tensor(delay))
    #print(output,output.shape,output.sum(-1),output.sum(-1).shape)
    output_str = out_to_str(output)
    return output_str


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [54]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(100):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    #print(s,d,result)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [63]:
import time
from torch.utils.data import DataLoader
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO
hidden_size = 18
model = VariableDelayGRUMemory(hidden_size,MAX_DELAY).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
batch_size = 64
Epochs = 100
train_dataloader = DataLoader(vds, batch_size)
for i in range(Epochs):
  print(i)
  for x,d,y in train_dataloader:
    x = idx_to_onehot(x)
    #y = idx_to_onehot(y)
    # logits batchsize*15*27 y batchsize*15
    x,d,y = x.to(device),d.to(device),y.to(device)
    logits, _ = model(x,d)
    loss = 0
    for j in range(d.shape[0]):
      a = logits[j,d[j]:SEQ_LENGTH,:]
      a.view(27,SEQ_LENGTH-d[j])
      b = y[j,d[j]:SEQ_LENGTH]
      b.view(SEQ_LENGTH-d[j])
      loss+=criterion(a,b)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(test_variable_delay_model(model),loss,time.time()-start_time) 


end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')


0
0.03927680798004988 tensor(52.1379, device='cuda:0', grad_fn=<AddBackward0>) 1.2493586540222168
1
0.048994974874371856 tensor(51.7949, device='cuda:0', grad_fn=<AddBackward0>) 2.465651035308838
2
0.07349246231155779 tensor(50.7147, device='cuda:0', grad_fn=<AddBackward0>) 3.719346046447754
3
0.136789506558401 tensor(47.4815, device='cuda:0', grad_fn=<AddBackward0>) 4.964017629623413
4
0.1334966319657073 tensor(44.9203, device='cuda:0', grad_fn=<AddBackward0>) 6.196865797042847
5
0.20649509803921567 tensor(39.8775, device='cuda:0', grad_fn=<AddBackward0>) 7.415040016174316
6
0.2201776649746193 tensor(34.9235, device='cuda:0', grad_fn=<AddBackward0>) 8.648558616638184
7
0.27467001885606535 tensor(33.8677, device='cuda:0', grad_fn=<AddBackward0>) 9.85156536102295
8
0.2808988764044944 tensor(32.5781, device='cuda:0', grad_fn=<AddBackward0>) 11.06050968170166
9
0.49906890130353815 tensor(23.4305, device='cuda:0', grad_fn=<AddBackward0>) 12.290926694869995
10
0.3807073954983923 tensor(28.6

In [65]:
test_variable_delay_model(model)

1.0